In [1]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm 

In [2]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/396008/hw_bootstrap.csv', sep=';', index_col = 'Unnamed: 0')

In [4]:
df

,value,experimentVariant
1,"10,3804953740268",Control
2,"9,54686666784264",Control
3,"11,0882146509904",Control
4,"10,1472740570122",Control
5,"9,78980767524929",Control
...,...,...
996,1000,Treatment
997,1200,Treatment
998,1500,Treatment
999,2000,Treatment


In [5]:
df.dtypes

value                object
experimentVariant    object
dtype: object

In [5]:
# меняем тип данных в столбце "value", а также меняем запятую на точку. 
df['value'] = df['value'].str.replace(',', '.').astype(float) 

In [6]:
df.isna().sum()

value                0
experimentVariant    0
dtype: int64

In [14]:
df[df['experimentVariant'] == 'Control'].shape[0]

500

In [15]:
df[df['experimentVariant'] == 'Treatment'].shape[0]

500

In [9]:
# function for mean 
def bootstrap_mean(
    data_column_1, 
    data_column_2, 
    boot_it = 1000, 
    statistic = np.mean, 
    conf_level = 0.95):
    boot_len = max([len(data_column_1), len(data_column_2)])
    boot_data = []
    
    for i in tqdm(range(boot_it)): # извлекаем подвыборки
        sample_1 = data_column_1.sample(boot_len, replace=True).values
        sample_2 = data_column_2.sample(boot_len, replace=True).values
        boot_data.append(statistic(sample_1 - sample_2))
    
    pd_boot_data = pd.DataFrame(boot_data) 
    
    # считаем доверительные интервалы
    left_quantile = (1 - conf_level) / 2
    right_quantile = 1 - (1 - conf_level) / 2
    ci = pd_boot_data.quantile([left_quantile, right_quantile])
    
    # считаем p-value
    p_1 = stats.norm.cdf(x = 0, loc = np.mean(boot_data), scale = np.std(boot_data))
    p_2 = stats.norm.cdf(x = 0, loc = - np.mean(boot_data), scale = np.std(boot_data))
    
    p_value = min(p_1, p_2) * 2
                
    return{'p_value' : p_value,
          'ci' : ci} 

In [10]:
# извлекаем необходимые наблюдения из исходного датафрейма
control = df[df['experimentVariant'] == 'Control'].value

In [11]:
# извлекаем необходимые наблюдения из исходного датафрейма
test = df[df['experimentVariant'] == 'Treatment'].value

In [12]:
# используем bootstrap, который показал нам, что при многократном повторении эксперимента и при истинности нулевой гипотезы
# (среднее в выборках не отличаются другу от друга), вероятность получить такие или более ярковыраженные различия составляет < 0.05,
# что позволяет нам отклонить нулевую гипотезу о равенстве средних. 
bootstrap_mean(control, test)

100%|██████████| 1000/1000 [00:00<00:00, 4049.33it/s]


{'p_value': 0.03327066570052403,
 'ci':                0
 0.025 -34.880654
 0.975  -2.918305}

In [13]:
# function for median 
def bootstrap_median(
    data_column_1, 
    data_column_2, 
    boot_it = 1000, 
    statistic = np.median, #в качестве описательной статистики используем median 
    conf_level = 0.95):
    boot_len = max([len(data_column_1), len(data_column_2)])
    boot_data = []
    
    for i in tqdm(range(boot_it)):
        sample_1 = data_column_1.sample(boot_len, replace=True).values
        sample_2 = data_column_2.sample(boot_len, replace=True).values
        boot_data.append(statistic(sample_1 - sample_2))
    
    pd_boot_data = pd.DataFrame(boot_data)
    
    
    left_quantile = (1 - conf_level) / 2
    right_quantile = 1 - (1 - conf_level) / 2
    ci = pd_boot_data.quantile([left_quantile, right_quantile])
    
    p_1 = stats.norm.cdf(x = 0, loc = np.mean(boot_data), scale = np.std(boot_data))
    p_2 = stats.norm.cdf(x = 0, loc = - np.mean(boot_data), scale = np.std(boot_data))
    
    p_value = min(p_1, p_2) * 2
                
    return{'p_value' : p_value,
          'ci' : ci} 

In [14]:
# если в качестве описательной статистики брать не mean, а median, 
# то полученный p-value позволяет нам принять нулевую гипотезу о равенстве средних в контрольной и тестовой группах. 
bootstrap_median(control, test)

100%|██████████| 1000/1000 [00:00<00:00, 3398.63it/s]


{'p_value': 0.8897931574101091,
 'ci':               0
 0.025 -0.155716
 0.975  0.137313}

In [15]:
# используем U тест Манна-Уитни о равенстве средних. Полученный p-value, который оказался больше > 0.05, 
# позволяет нам принять нулевую гипотезу о равенстве средних в контрольной и тестовой группах. 
stats.mannwhitneyu(control, test)

MannwhitneyuResult(statistic=124189.5, pvalue=0.42960742912427896)